### Here, we translate all the heavy atoms of our parameterized FMN gro/top to new positions specified by the docked model.

We also need to translate all the H-bonds as well, by parsing the topology file to find the bond vectors (C-H, N-H, O-H) 

**NOTE** The H positions will be all wonky, but at least they will be close enough to MINIMIZE without crashing.

In [9]:
%ls

FMN.amb2gmx/         FMN_fromMod.gro      leap.log
FMN.crd              FMN_fromMod.pdb      runme
FMN.off              README.md            translate_FMN.ipynb
FMN.pdb              build_FMN.leap
FMN.prmtop           fmn.frcfld


First, read in the grofile whose coords we will alter

In [10]:
grofile = 'FMN.amb2gmx/FMN_GMX.gro'
fin = open(grofile, 'r')
grolines = fin.readlines()
fin.close()

verbose = False
if (verbose):
    for line in lines:
        print(line.strip())

# Lines look like this:
"""
FMN_GMX.gro created by acpype (v: 2022.6.6) on Tue Apr  9 10:18:59 2024
50
1  FMN   N1    1  -0.832  -0.639  -0.053
1  FMN   C2    2  -0.898  -0.643  -0.173
1  FMN   O2    3  -1.018  -0.670  -0.181
...
1  FMN H5'1   49  -1.115  -0.785   0.436
1  FMN H5'2   50  -1.003  -0.658   0.490
28.05800     8.87600    17.95600
"""

import numpy as np

class Atom():
    """A Container object to store atom information and read/write representations."""
    
    def __init__(self, line):
        """Initialize by parsing the input line"""
        
        fields = (line.strip()).split()
        self.resnum    = int(fields[0])
        self.resname   = fields[1]
        self.atomname  = fields[2]
        self.atomnum   = int(fields[3])
        self.xyz       = np.array([float(fields[4]), float(fields[5]), float(fields[6])])
    
    def __repr__(self):
        """Return a representation of the atom -- a *.gro file line (without final newline)"""
        
        # f-string command to format and print the information
        return f"{self.resnum:>5d} {self.resname:>4s} {self.atomname:>4s} {self.atomnum:>4d} {self.xyz[0]:7.3f} {self.xyz[1]:7.3f} {self.xyz[2]:7.3f}"
        


In [11]:
# Read the grofile atoms into a dictionary
title_line   = grolines.pop(0)
natoms_line  = grolines.pop(0)
boxsize_line = grolines.pop()

atoms = {}  # dict of {atomnum: Atom()}
for line in grolines:
    a = Atom(line)
    print(a)
    atoms[a.atomnum] = a
    
heavy_atomnums = []
hydrogen_atomnums = []
for atomnum,a in atoms.items():
    if a.atomname.count('H'):
        hydrogen_atomnums.append(atomnum)
    else:
        heavy_atomnums.append(atomnum)
print('heavy_atomnums', heavy_atomnums)
print('hydrogen_atomnums', hydrogen_atomnums)

    1  FMN   N1    1  -0.832  -0.639  -0.053
    1  FMN   C2    2  -0.898  -0.643  -0.173
    1  FMN   O2    3  -1.018  -0.670  -0.181
    1  FMN   N3    4  -0.819  -0.615  -0.288
    1  FMN   C4    5  -0.682  -0.583  -0.290
    1  FMN   O4    6  -0.625  -0.561  -0.397
    1  FMN  C4A    7  -0.619  -0.581  -0.157
    1  FMN   N5    8  -0.485  -0.552  -0.150
    1  FMN  C5A    9  -0.423  -0.567  -0.030
    1  FMN   C6   10  -0.285  -0.556  -0.026
    1  FMN   C7   11  -0.217  -0.570   0.090
    1  FMN  C7M   12  -0.068  -0.556   0.089
    1  FMN   C8   13  -0.283  -0.596   0.206
    1  FMN  C8M   14  -0.207  -0.613   0.333
    1  FMN   C9   15  -0.419  -0.609   0.207
    1  FMN  C9A   16  -0.493  -0.595   0.090
    1  FMN  N10   17  -0.634  -0.606   0.084
    1  FMN  C10   18  -0.699  -0.610  -0.041
    1  FMN  C1'   19  -0.714  -0.557   0.203
    1  FMN  C2'   20  -0.749  -0.663   0.312
    1  FMN  O2'   21  -0.764  -0.600   0.438
    1  FMN  C3'   22  -0.870  -0.750   0.275
    1  FMN

In [12]:
topfile = 'FMN.amb2gmx/FMN_GMX.top'
fin = open(topfile, 'r')
toplines = fin.readlines()
fin.close()

hbond_pairs = []  # atomnum1, atomnum2
hbond_vectors = []  # the displacement vector from the heavy to the hydrogen

i = 0
while toplines[i].count('[ bonds ]') == 0:
    i += 1
i += 2  # skip this line, and skip the ; comment
while toplines[i].strip() != '':
    fields = toplines[i].split()
    atomnum1, atomnum2 = int(fields[0]), int(fields[1])  # note that the H is always the *second* number
    if (atomnum1 in hydrogen_atomnums) or (atomnum2 in hydrogen_atomnums):
        hbond_pairs.append( [atomnum1, atomnum2] )
        hbond_vectors.append( atoms[atomnum2].xyz - atoms[atomnum1].xyz )
    i += 1

print(hbond_pairs) 
print(hbond_vectors)


[[4, 32], [10, 33], [12, 34], [12, 35], [12, 36], [14, 37], [14, 38], [14, 39], [15, 40], [19, 41], [19, 42], [20, 43], [21, 44], [22, 45], [23, 46], [24, 47], [25, 48], [26, 49], [26, 50]]
[array([-0.045,  0.   , -0.091]), array([ 0.055,  0.021, -0.092]), array([0.033, 0.08 , 0.069]), array([ 0.049, -0.095,  0.03 ]), array([ 0.038,  0.026, -0.102]), array([-0.064, -0.044,  0.08 ]), array([ 0.086, -0.068, -0.012]), array([0.039, 0.098, 0.036]), array([-0.048, -0.021,  0.095]), array([-0.094,  0.051, -0.032]), array([0.059, 0.081, 0.05 ]), array([ 0.089, -0.066,  0.009]), array([-0.012, -0.071,  0.063]), array([ 0.02 , -0.037, -0.103]), array([-0.071, -0.054, -0.034]), array([ 0.012,  0.105, -0.034]), array([-0.09 ,  0.007,  0.036]), array([-0.037, -0.102,  0.024]), array([0.075, 0.025, 0.078])]


In [13]:
%ls FMN.amb2gmx

FMN_GMX.gro                             em.tpr
FMN_GMX.top                             em.trr
FMN_GMX_posre.itp                       ener.edr
FMN_GMX_posre.top                       index.ndx
FMN_GMX_translated.gro                  md.log
FMN_GMX_translated_minimized.gro        md.mdp
FMN_GMX_translated_minimized_nopbc.gro  mdout.mdp
acpype.log                              out.gro
em.edr                                  posre_FMN.itp
em.gro                                  rungmx.sh*
em.log                                  rungmx_wcoords.sh*
em.mdp                                  traj.trr


Next, we will read it the grofile that contains the target coords

In [14]:
grofile_wcoords = 'FMN_fromMod.gro'
fin = open(grofile_wcoords, 'r')
grofile_wcoords_lines = fin.readlines()
fin.close()

# This files looks like this:
"""Glycine aRginine prOline Methionine Alanine Cystine Serine
31
3609FMN      P    1   6.175  12.325   3.811
3609FMN    O1P    2   6.164  12.358   3.954
3609FMN    O2P    3   6.091  12.191   3.780
...
3609FMN    C10   30   5.852  13.129   3.767
3609FMN    N10   31   5.956  13.103   3.683
0.00000   0.00000   0.00000
"""

# The lack of a space in '3609FMN' will break the Atom() initializer, so let's put a space in there:
for i in range(len(grofile_wcoords_lines)):
    grofile_wcoords_lines[i] = grofile_wcoords_lines[i].replace('3609FMN', '3609 FMN')

# Read the grofile_wcoords atoms into a dictionary
atoms_wcoords = {}  # dict of {atomnum: Atom()}
for line in grofile_wcoords_lines[2:-1]:
    a = Atom(line)
    atoms_wcoords[a.atomnum] = a

print(atoms_wcoords)

{1:  3609  FMN    P    1   6.175  12.325   3.811, 2:  3609  FMN  O1P    2   6.164  12.358   3.954, 3:  3609  FMN  O2P    3   6.091  12.191   3.780, 4:  3609  FMN  O3P    4   6.329  12.301   3.773, 5:  3609  FMN  C5'    5   6.194  12.561   3.754, 6:  3609  FMN  O5'    6   6.116  12.446   3.723, 7:  3609  FMN  C4'    7   6.143  12.681   3.673, 8:  3609  FMN  O4'    8   6.259  12.763   3.693, 9:  3609  FMN  C3'    9   6.023  12.743   3.585, 10:  3609  FMN  O3'   10   5.955  12.655   3.496, 11:  3609  FMN  C2'   11   5.953  12.880   3.584, 12:  3609  FMN  O2'   12   5.816  12.863   3.622, 13:  3609  FMN  C1'   13   6.022  12.972   3.684, 14:  3609  FMN   N1   14   5.806  13.039   3.853, 15:  3609  FMN   C2   15   5.704  13.065   3.934, 16:  3609  FMN   O2   16   5.667  12.977   4.010, 17:  3609  FMN   N3   17   5.640  13.182   3.938, 18:  3609  FMN   C4   18   5.675  13.285   3.856, 19:  3609  FMN  C4A   19   5.788  13.261   3.763, 20:  3609  FMN   O4   20   5.616  13.391   3.859, 21:  360

### Copy the target coords of the heavy atoms ...
### ... then hydrogens into the grofile


In [15]:
for atomnum in heavy_atomnums:
    name = atoms[atomnum].atomname
    
    # Look for this atom in the atoms_wcoords
    for atomnum_wcoords, a_wcoords in atoms_wcoords.items():
        if a_wcoords.atomname == name:
            atoms[atomnum].xyz = a_wcoords.xyz
            print(atoms[atomnum])
            break

# ... then the hydrogens
for i in range(len(hbond_pairs)):
    atomnum1, atomnum2 = hbond_pairs[i]
    atoms[atomnum2].xyz = atoms[atomnum1].xyz + hbond_vectors[i]
    print(atoms[atomnum2])


    1  FMN   N1    1   5.806  13.039   3.853
    1  FMN   C2    2   5.704  13.065   3.934
    1  FMN   O2    3   5.667  12.977   4.010
    1  FMN   N3    4   5.640  13.182   3.938
    1  FMN   C4    5   5.675  13.285   3.856
    1  FMN   O4    6   5.616  13.391   3.859
    1  FMN  C4A    7   5.788  13.261   3.763
    1  FMN   N5    8   5.830  13.353   3.680
    1  FMN  C5A    9   5.933  13.328   3.598
    1  FMN   C6   10   5.977  13.429   3.509
    1  FMN   C7   11   6.081  13.404   3.425
    1  FMN  C7M   12   6.128  13.512   3.330
    1  FMN   C8   13   6.144  13.280   3.425
    1  FMN  C8M   14   6.259  13.255   3.330
    1  FMN   C9   15   6.103  13.180   3.510
    1  FMN  C9A   16   5.997  13.203   3.598
    1  FMN  N10   17   5.956  13.103   3.683
    1  FMN  C10   18   5.852  13.129   3.767
    1  FMN  C1'   19   6.022  12.972   3.684
    1  FMN  C2'   20   5.953  12.880   3.584
    1  FMN  O2'   21   5.816  12.863   3.622
    1  FMN  C3'   22   6.023  12.743   3.585
    1  FMN

### Write the result to file, using the *receptor's* box size

In [16]:
out_grofile = 'FMN.amb2gmx/FMN_GMX_translated.gro'
fout = open(out_grofile, 'w')
fout.write(title_line)
fout.write(natoms_line)
for atomnum in range(1, 51):
    fout.write(repr(atoms[atomnum])+'\n')
    
fout.write('   4.83860   6.38259   6.47657\n')  # from ../receptor/conf.gro    
# fout.write(boxsize_line)
fout.close()
print(f'Wrote: {out_grofile}')



Wrote: FMN.amb2gmx/FMN_GMX_translated.gro
